In [144]:
import yfinance as yf
import pandas as pd 
import numpy as np
import ta
import matplotlib.pyplot as plt
import pandas_ta as ta1


In [145]:
df = yf.download('EURUSD=X',interval='1d')

[*********************100%%**********************]  1 of 1 completed


In [146]:
df['ma_20'] = df.Close.rolling(20).mean()
df['vol'] = df.Close.rolling(20).std()
df['upper_bb'] = df['ma_20'] + (df['vol'] * 2)
df['lower_bb'] = df['ma_20'] - (df['vol'] * 2)


In [147]:
df['rsi'] = ta.momentum.rsi(df['Close'], window=14)

In [160]:




df['%K'] = ta.momentum.stoch(df['High'], df['Low'], df['Close'], window=14, smooth_window=3)
df['%D'] = df['%K'].rolling(3).mean()
df['macd'] = ta.trend.macd_diff(df['Close'], window_slow=26, window_fast=12, window_sign=9)

In [161]:
df.dropna(inplace=True)

In [150]:
conditions = [
    (df.Close < df.lower_bb) & (df.rsi < 30), 
    (df.Close > df.upper_bb) & (df.rsi > 30)
]

choices = ['Buy','Sell']

df['Signal'] = np.select(conditions, choices)

In [151]:
df.dropna(inplace=True)

In [152]:
df['Signal'] = df.Signal.shift()

In [153]:
df['shifted_close'] = df['Close'].shift()

In [154]:
def Backtest(df):
    position = False
    buydates = []
    buyprices = []
    selldates = []
    sellprices = []
    
    for index, row in df.iterrows():
        if row['Signal'] == 'Buy' and not position:
            position = True
            buydates.append(index)
            buyprices.append(row['Close'])
        
        if position:
            if row['Signal'] == 'Sell' or row['shifted_close'] < 0.95 * buyprices[-1]:
                selldates.append(index)
                sellprices.append(row['Close'])
                position = False

    # Ensure lists are of equal length by filling with NaN
    max_len = max(len(buydates), len(selldates))
    buydates.extend([np.nan] * (max_len - len(buydates)))
    buyprices.extend([np.nan] * (max_len - len(buyprices)))
    selldates.extend([np.nan] * (max_len - len(selldates)))
    sellprices.extend([np.nan] * (max_len - len(sellprices)))
    
    return buydates, buyprices, selldates, sellprices


In [155]:
# Assuming you have already defined and run the Backtest function
buydates, buyprices, selldates, sellprices = Backtest(df)

# Create a trade history dictionary
trade_history = {
    'Buy Date': buydates,
    'Buy Price': buyprices,
    'Sell Date': selldates,
    'Sell Price': sellprices,
}

# Convert the dictionary to a DataFrame
trade_history_df = pd.DataFrame(trade_history)


# Display the DataFrame
print(trade_history_df)

     Buy Date  Buy Price  Sell Date  Sell Price
0  2005-02-08   1.276992 2005-02-23    1.320306
1  2005-06-01   1.219602 2005-08-04    1.238497
2  2008-08-26   1.464794 2008-10-03    1.378493
3  2008-10-07   1.358806 2008-10-23    1.294498
4  2009-12-18   1.434905 2010-01-14    1.449990
5  2010-02-01   1.392699 2010-03-15    1.367746
6  2010-05-06   1.263296 2010-06-07    1.192293
7  2010-06-08   1.197203 2010-07-16    1.293193
8  2011-09-13   1.366027 2011-12-30    1.295605
9  2012-01-10   1.277596 2012-02-27    1.346711
10 2012-05-17   1.273075 2012-07-26    1.214034
11 2014-08-22   1.327933 2014-10-07    1.264606
12 2015-01-06   1.193902 2015-01-27    1.124771
13 2015-03-09   1.084599 2015-05-01    1.121466
14 2016-11-16   1.072731 2017-01-19    1.062778
15 2018-05-03   1.195100 2018-08-16    1.134430
16 2019-09-04   1.097454 2019-10-21    1.115573
17 2020-02-12   1.091941 2020-03-04    1.118068
18 2020-03-23   1.069439 2020-05-22    1.095290
19 2021-03-10   1.189791 2021-05-11    1

In [156]:
trade_history_df['Return'] = (trade_history_df['Sell Price'] - trade_history_df['Buy Price']) / trade_history_df['Buy Price']

In [157]:
trade_history_df['Days'] = (trade_history_df['Sell Date'] - trade_history_df['Buy Date'])

In [158]:
trade_history_df['PnL'] = trade_history_df['Sell Price'] - trade_history_df['Buy Price']

In [159]:
trade_history_df

,Buy Date,Buy Price,Sell Date,Sell Price,Return,Days,PnL
0,2005-02-08,1.276992,2005-02-23,1.320306,0.033919,15 days,0.043314
1,2005-06-01,1.219602,2005-08-04,1.238497,0.015493,64 days,0.018896
2,2008-08-26,1.464794,2008-10-03,1.378493,-0.058917,38 days,-0.086301
3,2008-10-07,1.358806,2008-10-23,1.294498,-0.047327,16 days,-0.064308
4,2009-12-18,1.434905,2010-01-14,1.449990,0.010512,27 days,0.015084
5,2010-02-01,1.392699,2010-03-15,1.367746,-0.017918,42 days,-0.024954
6,2010-05-06,1.263296,2010-06-07,1.192293,-0.056205,32 days,-0.071003
7,2010-06-08,1.197203,2010-07-16,1.293193,0.080178,38 days,0.095989
8,2011-09-13,1.366027,2011-12-30,1.295605,-0.051552,108 days,-0.070422
9,2012-01-10,1.277596,2012-02-27,1.346711,0.054097,48 days,0.069115
